In [1]:
from lib.solver_tools.solver_client import SolverClient
from lib.solver_tools import Task
from lib.solver_tools.utils import make_task_comparison_link
import json
from copy import deepcopy
from lib.common_tools.read_token import read_solver_apikey
import os

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import vrp_algorithms_lib.common_tools.file_utils as file_utils
from vrp_algorithms_lib.problem.models import Routes
from typing import Any, Dict

In [3]:
my_apikey = read_solver_apikey()
solver_client = SolverClient(my_apikey)

In [4]:
def get_request_with_fixed_routes(request: Dict[str, Any], routes: Routes) -> Dict[str, Any]:
    # routes must contain fields 'vehicle_id', 'location_ids'
    new_request = deepcopy(request)
    vehicle_id_to_location_ids = {route.vehicle_id : route.location_ids for route in routes.routes}
    for vehicle in new_request['vehicles']:
        vehicle['visited_locations'] = [{'id': location_id} for location_id in vehicle_id_to_location_ids[vehicle['id']]]
    new_request['options']['solver_time_limit_s'] = 0
    return new_request

In [5]:
def get_task_id_with_fixed_routes(path_to_request: str, path_to_routes: str, solver_client: SolverClient):
    request = file_utils.read_json(path_to_request)
    
    routes_json = file_utils.read_json(path_to_routes)
    routes: Routes = Routes.parse_obj(routes_json)
        
    request_with_fixed_routes = get_request_with_fixed_routes(request, routes)
    task_id_with_fixed_routes = solver_client.add_task_and_get_task_id(request_with_fixed_routes)
    return task_id_with_fixed_routes

In [6]:
path_to_request = '../test_data/inputs/large_test_2/request.json'
path_to_routes_annealing = os.path.join('../test_data/results/annealing/large_test_2/30000000_iterations_routes_1.json')
task_id_annealing = get_task_id_with_fixed_routes(path_to_request, path_to_routes_annealing, solver_client)

/b2bgeo/environments/common_env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'courier.yandex.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
Task(task_id_annealing).map()

Neither solver_api_key nor ya_courier_token provided to SolverClient. Using client in limited functionality mode


ValueError: Task 85a839b0-35334593-180a978d-45c30fbf hasn't been solved yet. ETA 00:32:34

In [8]:
path_to_routes_greedy = os.path.join('../test_data/results/greedy/large_test_2/routes.json')
task_id_greedy = get_task_id_with_fixed_routes(path_to_request, path_to_routes_greedy, solver_client)

/b2bgeo/environments/common_env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'courier.yandex.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
Task(task_id_greedy).map()

Neither solver_api_key nor ya_courier_token provided to SolverClient. Using client in limited functionality mode


ValueError: Task c91d73b6-af9ba725-735b627-80d69e24 hasn't been solved yet. ETA 00:32:41

In [10]:
task_id_greedy

'c91d73b6-af9ba725-735b627-80d69e24'

In [11]:
make_task_comparison_link(
    ['85a839b0-35334593-180a978d-45c30fbf', 'c91d73b6-af9ba725-735b627-80d69e24'],
    ['Annealing', 'Greedy']
)